In [33]:
import pandas as pd
import numpy as np

In [34]:
df = pd.read_csv('pacific.csv')

In [35]:
df.replace(-999, np.nan, inplace=True)

In [36]:
def convert_lat_long(val):
    # removes the direction (n/s/e/w) and converts to float, negates if south or west
    direction = val[-1]
    val = float(val[:-1])
    if direction in ['S', 'W']:
        val *= -1
    return val

In [37]:
df['Latitude'] = df['Latitude'].apply(convert_lat_long)
df['Longitude'] = df['Longitude'].apply(convert_lat_long)

In [38]:
# parsing dates and times to combine into a datetime column
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')
df['Time'] = df['Time'].apply(lambda x: "{:04d}".format(x))  # pads to be 4 digits
df['Time'] = pd.to_timedelta(df['Time'].str[:2] + ':' + df['Time'].str[2:] + ':00')  # converts to timedelta (duration / diff between 2 dates/tiems)
df['DateTime'] = df['Date'] + df['Time']
df.drop(['Date', 'Time'], axis=1, inplace=True)

In [39]:
# ensure numeric columns are of the correct data type
numeric_cols = ['Maximum Wind', 'Minimum Pressure', 'Low Wind NE', 'Low Wind SE', 'Low Wind SW', 'Low Wind NW', 'Moderate Wind NE', 'Moderate Wind SE', 'Moderate Wind SW', 'Moderate Wind NW', 'High Wind NE', 'High Wind SE', 'High Wind SW', 'High Wind NW']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [40]:
print(df.head(), df.size)

         ID                 Name Event Status  Latitude  Longitude  \
0  EP011949              UNNAMED           TS      20.2     -106.3   
1  EP011949              UNNAMED           TS      20.2     -106.4   
2  EP011949              UNNAMED           TS      20.2     -106.7   
3  EP011949              UNNAMED           TS      20.3     -107.7   
4  EP011949              UNNAMED           TS      20.4     -108.6   

   Maximum Wind  Minimum Pressure  Low Wind NE  Low Wind SE  ...  Low Wind NW  \
0            45               NaN          NaN          NaN  ...          NaN   
1            45               NaN          NaN          NaN  ...          NaN   
2            45               NaN          NaN          NaN  ...          NaN   
3            45               NaN          NaN          NaN  ...          NaN   
4            45               NaN          NaN          NaN  ...          NaN   

   Moderate Wind NE  Moderate Wind SE  Moderate Wind SW  Moderate Wind NW  \
0              

In [41]:
missing_data_threshold = 0.2  # 20% threshold for removal
imputation_threshold = 0.8   # 80% threshold for imputation

In [42]:
# Calculate the percentage of missing values for each hurricane (ID)
missing_percentage_by_id = df.isnull().groupby(df['ID']).mean().max(axis=1)

# Find IDs with missing data above the threshold
ids_to_remove = missing_percentage_by_id[missing_percentage_by_id > missing_data_threshold].index

# Remove those IDs from the DataFrame
df = df[~df['ID'].isin(ids_to_remove)]

# Now, for the remaining data, impute missing values where the missing data is below the threshold
# Define a function to impute missing values for a column
def impute_missing(df, column):
    # Calculate the mean for the column, ignoring NaN values
    mean_value = df[column].mean()
    
    # Only impute if the percentage of NaN is below the threshold
    if df[column].isnull().mean() < missing_data_threshold:
        df[column].fillna(mean_value, inplace=True)

# Apply the imputation function to each numeric column
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    impute_missing(df, col)

In [43]:
print(df.head(), df.size)

             ID                 Name Event Status  Latitude  Longitude  \
19744  EP012004               AGATHA           TD      14.7     -107.6   
19745  EP012004               AGATHA           TD      15.4     -108.5   
19746  EP012004               AGATHA           TS      16.0     -109.1   
19747  EP012004               AGATHA           TS      16.6     -109.6   
19748  EP012004               AGATHA           TS      17.1     -109.9   

       Maximum Wind  Minimum Pressure  Low Wind NE  Low Wind SE  ...  \
19744            25            1006.0          0.0          0.0  ...   
19745            30            1006.0          0.0          0.0  ...   
19746            35            1005.0         75.0         75.0  ...   
19747            45            1000.0         75.0         75.0  ...   
19748            50             997.0         75.0         75.0  ...   

       Low Wind NW  Moderate Wind NE  Moderate Wind SE  Moderate Wind SW  \
19744          0.0               0.0          